In [8]:
!pip install transformers datasets evaluate rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /system/linux/mambaforge-3.11/lib/python3.11/site-packages/matlabengineforpython-9.14-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5490276cf743e6f0c6021942ee4fb6cd96eb5f934a15b8e8499b52de8b5fb246
  Stored in directory: /tmp/.xdg_cache_phudish_p/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
from datasets import load_dataset
billsum = load_dataset("billsum", split="ca_test")
billsum = billsum.train_test_split(test_size=0.2)

In [16]:
from transformers import AutoTokenizer
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [17]:
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [19]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [20]:
import evaluate
rouge = evaluate.load("rouge")

In [21]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(f"Predictions : {predictions}")
    print(f"Labels : {labels}")
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [22]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.864576,0.129100,0.038100,0.106700,0.107100,19.000000


/accounts/grad/phudish_p/.local/lib/python3.11/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predictions : [[    0     8   973 ...     7  3636     3]
 [    0     8   958 ... 15421    19 17180]
 [    0  5568 20236 ... 20236 18189     5]
 ...
 [    0     8   538 ... 10005    19 21012]
 [    0     8  3786 ...     9    61   432]
 [    0  5568   668 ...     3     5    37]]
Labels : [[17061    53   973 ...     5   100     1]
 [ 1881   155    53 ...    92 27765     1]
 [17061    53   973 ...   169     6     1]
 ...
 [17061    53   973 ...  1045 11069     1]
 [   37 21171   283 ... 26533  1393     1]
 [17061    53   973 ...  1356    24     1]]


TrainOutput(global_step=62, training_loss=3.186487013293851, metrics={'train_runtime': 24.8355, 'train_samples_per_second': 39.822, 'train_steps_per_second': 2.496, 'total_flos': 267706083311616.0, 'train_loss': 3.186487013293851, 'epoch': 1.0})